In [1]:
%cd "~/repositories/clim-agg"

/home3/dth2133/repositories/clim-agg


In [2]:
import xarray as xr
from aggregate import ClimateDataset, GridWeights, GeoRegions
from aggregate import SpatialAggregator, TemporalAggregator
from aggregate.gridfuncs import *
from aggregate.utils import *
from dask.distributed import Client
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [107]:
# Set file output name/path
output_path = "/home3/dth2133/data/aggregated/counties/"
output_name = "usa_counties_corn"
output_varn = "gdd8_30"
csv = True

In [103]:
# Open shapefile containing region features.
georegions = GeoRegions.from_name('counties')

# Open example climate dataset to calculate grid weights.
clim = ClimateDataset.from_path(
    "/home3/dth2133/data/usa/usa-t2m_tempPrecLand2019.zarr", 
    't2m', 
    'zarr', 
    preprocess=lambda x: x - 273.15)

# Clip climate data to the US (raw data are global)
clim.clip_data_to_georegions_extent(georegions)

# Calculate area and crop layer weights.
weights = GridWeights.from_objects(clim, georegions, crop='corn')

# This object covers aggregating hourly and daily data to the yearly 
# level
temporal = TemporalAggregator.from_name(name='era5l',
              calc={'daily':('dd', 1, [8,30,0]),
                    'yearly':('sum', 1)})
# temporal = TemporalAggregator.from_name(name='era5l',
#               calc={'daily':('avg', 1),
#                     'yearly':('avg', 1)})

# This object aggregates cells within a region to the average across 
# cells, weighted by `weights`, which in this case are the area of the
# cell and the share of the cell with corn crops.
spatial = SpatialAggregator.from_name('era5l')

In [5]:
# Calculate the grid weights
w = weights.weights()

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed | 24.4s
[########################################] | 100% Completed | 23.8s
[########################################] | 100% Completed | 43.3s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s


In [104]:
def aggregate_era5l_t2m(year):
    
    # Open climate dataset.
    clim = ClimateDataset.from_path(
        f"/home3/dth2133/data/usa/usa-t2m_tempPrecLand{year}.zarr", 
        't2m', 
        'zarr', 
        preprocess=lambda x: x - 273.15) # Kelvin to Celsius

    # Clip climate data to the US (raw data are global)
    clim.clip_data_to_georegions_extent(georegions)
    # Rechunk dataset to optimize multithreading
    clim.rechunk((5, 578, -1, -1, -1))

    # Update climate dataset in `clim` to one collapsed over
    # hour and day based upon `temporal` definition above.
    clim.update(
        clim.da.data.map_blocks(
            temporal.execute,
            dtype=float,
            drop_axis=(3,4)),
        drop_dims=['day', 'hour'])
    clim.rechunk(-1)

    # Again update, but with (weighted) spatial collapse over regions.
    clim.update(
        w.data.map_blocks(
            spatial.execute,
            clim.da.data,
            dtype=float,
            drop_axis=[1,2],
            new_axis=1),
        drop_dims=['latitude', 'longitude'],
        new_dims={'region': w.region.values})

    # Send back the aggregated climate data.
    return clim.da


In [105]:
# Check years from the input path - this just creates a vector of years for
# which my raw climate data are available, e.g. [1970, 1971, ...]
import glob
years = np.sort([x[44:-5] for x in glob.glob('/home3/dth2133/data/usa/*t2m*')])
# Loop over years and aggregate.
output = list()
for year in years:
    print(year)
    output.append(aggregate_era5l_t2m(year))


1951
1952
1953
1954
1955
1956
1957
1958
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [106]:
# Combine all our data
da = xr.concat(output, dim='year').compute()
da

[########################################] | 100% Completed | 10min 41.2s


<xarray.DataArray 'execute-e927e0e5e9f8b9c867776967587a50f1' (region: 3108, year: 68)>
array([[2967.06443571, 2666.46259859, 2818.68003309, ..., 3414.17380564,
        3005.9233624 , 3092.50321413],
       [ 456.41040935,  428.42894714,  460.95978929, ...,  513.0914094 ,
         490.07844835,  482.33758735],
       [2058.02834974, 1848.44145326, 1992.11336381, ..., 2289.40600142,
        2141.31402882, 2053.58703216],
       ...,
       [ 229.78707099,  226.34144134,  221.62452345, ...,  256.67351462,
         227.26663547,  223.7805885 ],
       [ 174.92529321,  192.84603674,  202.35727942, ...,  187.6676883 ,
         183.30998011,  170.94427792],
       [2236.38701398, 2315.37338724, 2430.50924258, ..., 2476.04198291,
        2478.19049171, 2322.95191886]])
Coordinates:
  * region   (region) object '21007' '21017' '21031' ... '48171' '55079' '26139'
  * year     (year) int64 1951 1952 1953 1954 1955 ... 2015 2016 2017 2018 2019

In [108]:
ds = da.to_dataset(name=output_varn)
ds.to_zarr(os.path.join(output_path, output_name+'.zarr'), mode='a') 

In [109]:
if csv:
    ds = xr.open_zarr(os.path.join(output_path, output_name+'.zarr'))
    ds.to_dataframe().to_csv(os.path.join(output_path, output_name+'.csv'))

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
